In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import asyncio

In [12]:
class State(TypedDict): # create a type bound dictionary type 

    messages : Annotated[list , add_messages(format="langchain-openai")] # create a key in the dict type with additional metadata added using Annotated
    # This basically means that it instructs langgraph to update message using the add_message function
    # function like add_message are called reducers

graph = StateGraph(State)

Now our chatbot can do two key things:
- Each node can take the state variable as input and output an update to the state variable
- Every time a node updates the message list of the state variable the messages are appended instead of just overwriting over the previous messages.

In [3]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

model  = init_chat_model(model="deepseek/deepseek-chat-v3-0324:free" , model_provider="openai")

# creating the node function
def llm(state: State):
    return {"messages" : [model.invoke(state["messages"])]}

# adding the node 
# the first argument is the unique node name 
# second argument is the callback function whenever the node is called
graph.add_node("llm" , llm)

In [4]:
graph.add_edge(START , "llm") # tell the graph where to start 
graph.add_edge("llm" ,END) # tell the graph where to end i.e. anytime this node is run you can exit

In [5]:
compiledGraph = graph.compile() # compile the whole graph to use invoke in the script

#### Now building the chatbot

In [6]:
# print (compiledGraph.invoke({"messages": [{"role": "user", "content": "lets so something fun"}]})) # testing

In [16]:
# async def chat():
#     while True:
#         inp = input("User: ")
#         print("Assistant: ",compiledGraph.invoke({
#             "messages" : [
#                 {"role":"user" , "content": inp},
#                 ]
#             })["messages"][-1].content)

# # asyncio.run(chat())
# await chat()

def stream_graph_updates(user_input: str):
    print("Assitant: ", len(
        compiledGraph.invoke(
        {
            "messages": [
                {"role": "user", "content": user_input},
            ]
        }
    )["messages"]
    ))


while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    stream_graph_updates(user_input)

Assitant:  2
Assitant:  2
Goodbye!
